In [1]:
### This script will automatically detect ZDR arcs (and KDP feet) in WSR-88D radar data
import matplotlib.pyplot as plt
import pyart
import numpy as np
import numpy.ma as ma
from metpy.units import atleast_1d, check_units, concatenate, units
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from siphon.radarserver import RadarServer
#rs = RadarServer('http://thredds-aws.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
#rs = RadarServer('http://thredds.ucar.edu/thredds/radarServer/nexrad/level2/IDD/')
from datetime import datetime, timedelta
from siphon.cdmr import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from metpy.units import atleast_1d, check_units, concatenate, units
from shapely.geometry import polygon as sp
import pyproj 
import shapely.ops as ops
from shapely.ops import transform
from shapely.geometry.polygon import Polygon
from functools import partial
from shapely import geometry
import netCDF4
from scipy import ndimage as ndi
#from skimage.feature import peak_local_max
#from skimage import data, img_as_float
from pyproj import Geod
from metpy.calc import get_wind_dir, get_wind_speed, get_wind_components
import matplotlib.lines as mlines
import pandas as pd
import scipy.stats as stats
import csv
import pickle
from sklearn.ensemble import RandomForestClassifier
import nexradaws
import os


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\pyart\graph\cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\pyart\graph\cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\babel\plural.py:301: DeprecationWarning: Flags not at the start of the expression \s+(?u)
  (None, re.compile(r'\s+(?u)')),


In [2]:
#Bring in Homeyer rainbow colormap. This line will not be needed once the next version of PyART comes out, 
#but is commented out here since I'm defaulting it to run without it at the moment.
#%run code_colormaps_CVD/colormap_generator.py

In [3]:
#Adding arrays containing the algorithm inputs for several tornadic supercell cases as examples
#FFD angle, determined visually as the angle of the reflectivity gradient vector along the FFD
storm_relative_dirstm = np.asarray([180, 200, 225, 190, 225, 190, 180, 190, 160, 170, 190, 165, 150])
#ZDR value defining the ZDR arc 'core', here set to 3.25 dB
zdrlevstm = np.asarray([3.25, 3.25, 3.25, 3.25, 3.25, 3.25, 3.25, 3.25, 3.25, 3.25, 3.25, 3.25, 3.25])
#KDP value defining the edge of the KDP foot. All sectiond dealing with KDP are still in the early stages of development
kdplevstm = np.asarray([1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5])
#First reflectivity contour used in the storm tracking algorithm
REFlevstm = np.asarray([43, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45])
#Second reflectivity contour used in the storm tracking algorithm
REFlev1stm = np.asarray([48, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50])
#Storm area threshold at which the tracking algorithm starts looking for more intense embedded cores
big_stormstm = np.asarray([300, 300, 300, 300, 300, 300, 300, 600, 300, 300, 300, 300, 300])
#Value to get rid of bad data files from THREDDS server, not really needed anymore
zero_z_triggerstm = np.asarray([25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25])
#Number of storm to track for earlier machine learning algorithm validation experiment, 
#not needed for just running the algorithm (just set to any integer)
storm_to_trackstm = np.asarray([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])
#Year of supercell case start time
yearstm = np.asarray([2017, 2016, 2017, 2017, 2015, 2018, 2016, 2014, 2014, 2017, 2016, 2018, 2015])
#Month of case start time
monthstm = np.asarray([4, 8, 2, 7, 7, 6, 6, 6, 5, 5, 5, 5, 11])
#Day of case start time
daystm = np.asarray([2, 27, 7, 11, 14, 11, 13, 18, 11, 18, 24, 29, 16])
#Hour of case start time (in UTC)
hourstm = np.asarray([17, 19, 20, 22, 0, 21, 22, 2, 19, 18, 22, 20, 22])
#Minute of case start time
start_minstm = np.asarray([30, 30, 30, 0, 0, 30, 0, 30, 55, 17, 18, 35, 25])
#Duration of analysis period, in hours
durationstm = np.asarray([2.0, 2.1, 2.2, 3.3, 3.0, 1.7, 3.6, 2.6, 1.7, 1.9, 1.5, 2.1, 2.7])
#WSR-88D radar site
stationstm = ['KPOE', 'KMVX', 'KDGX', 'KMVX', 'KIND', 'KOAX', 'KAMA', 'KFSD', 'KUEX', 'KFDR', 'KDDC', 'KDDC', 'KAMA']

Checklist of things to go through to make sure the algorithm will run on a particular computer:
1. Specify a folder to direct the radar downloads to (line 34 in the cell below)
2. Make sure the saved Random Forest file is in the directory this script is running in


In [4]:
#Actual code for the ZDR arc algorithm
def multi_case_algorithm_ML1(storm_relative_dir, zdrlev, kdplev, REFlev, REFlev1, big_storm, zero_z_trigger, storm_to_track, year, month, day, hour, start_min, duration, station):    
    #Settings
    #Set vector perpendicular to FFD Z gradient
    storm_relative_dir = storm_relative_dir
    #Set ZDR Threshold for outlining arcs
    zdrlev = [zdrlev]
    #Set KDP Threshold for finding KDP feet
    kdplev = [kdplev]
    #Set reflectivity thresholds for storm tracking algorithm
    REFlev = [REFlev]
    REFlev1 = [REFlev1]
    #Set storm size threshold that triggers subdivision of big storms
    big_storm = big_storm #km^2
    #Set search radii around storm centroids for ZDR arc objects. If fixed threshold is needed, uncomment these and comment
    #out the lines where these variables are set in the algorithm
    Outer_r = 30 #km
    Inner_r = 6 #km
    #Set trigger to ignore strangely-formatted files right before 00Z
    #Pre-SAILS #: 17
    #SAILS #: 25
    zero_z_trigger = zero_z_trigger

    storm_to_track = storm_to_track

    #Here, set the initial time of the archived radar loop you want.
    dt = datetime(year,month, day, hour, start_min) # Our specified time
    station = station
    end_dt = dt + timedelta(hours=duration)
    #Set up nexrad interface
    conn = nexradaws.NexradAwsInterface()
    scans = conn.get_avail_scans_in_range(dt,end_dt,station)
    #Change this to whatever folder you want the radar data to download into.
    results = conn.download(scans, 'RadarFolder')
    #query = rs.query()
    #Set the duration of the loop in hours
    #query.stations(station).time_range(dt, dt + timedelta(hours=duration))
    #cat = rs.get_catalog(query)
    #cat.datasets
    
    #Create an option for just reading all files in a folder
    #folder = 'May27KDDC'

    #Setting counters for figures and Pandas indices
    f = 27
    n = 1
    storm_index = 0
    scan_index = 0
    #Create geod object for later distance and area calculations
    g = Geod(ellps='sphere')
    #Open the placefile
    f = open("ARCexample"+station+str(dt.year)+str(dt.month)+str(dt.day)+str(dt.hour)+str(dt.minute)+"_Placefile.txt", "w+")
    f.write("Title: ZDR Arc Placefile \n")
    f.write("Refresh: 8 \n \n")
    ####
    #Load ML algorithm
    #Make surt that you have this file in the directory that this script is in.
    forest_loaded = pickle.load(open('BestRandomForest.pkl', 'rb'))
    #Create file for ML algorithm test/training data
    #Tornadic filename
    #with open('Machine_Learning/ML_test'+station+str(dt.year)+str(dt.month)+str(dt.day)+str(dt.hour)+str(dt.minute)+'.csv', 'w') as csvfile:
    #Nontornadic filename
    #with open('Machine_Learning/NT2_ML_test'+station+str(dt.year)+str(dt.month)+str(dt.day)+str(dt.hour)+str(dt.minute)+'.csv', 'w') as csvfile:
    #    fieldnames = ['number', 'hour', 'minute','area','distance','angle','mean','max','mean_cc','mean_kdp','mean_Z','mean_graddir','mean_grad', 'raw_angle']
    #    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    #    writer.writeheader()
    #print(datetime.utcnow())
    ####
    #Actual algorithm code starts here
    #for item in sorted(cat.datasets.items()):
    for i,scan in enumerate(results.iter_success(),start=1):
    #Local file option:
    #for radar_file in os.listdir(folder):
        #print(radar_file)
        #Loop over all files in the dataset and pull out each 0.5 degree tilt for analysis
        try:
            #ds = item[1]
            radar1 = scan.open_pyart()
            #Local file option
            #radar1 = pyart.io.nexrad_archive.read_nexrad_archive(folder+'/'+radar_file)
            #print('file read')
            #Make sure the file isn't a strange format
            if radar1.nsweeps > zero_z_trigger:
                continue
            for i in range(radar1.nsweeps):
                print('in loop')
                print(radar1.nsweeps)
                radar = radar1.extract_sweeps([i])
                #Checking to make sure the tilt in question has all needed data and is the right elevation
                if ((np.mean(radar.elevation['data']) < .65) and (np.max(np.asarray(radar.fields['differential_reflectivity']['data'])) != np.min(np.asarray(radar.fields['differential_reflectivity']['data'])))):
                    n = n+1
                    print(np.mean(radar.elevation['data']))
                    time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
                    object_number=0.0
                    print(time_start)
                    month = time_start.month
                    if month < 10:
                        month = '0'+str(month)
                    hour = time_start.hour
                    if hour < 10:
                        hour = '0'+str(hour)
                    minute = time_start.minute
                    if minute < 10:
                        minute = '0'+str(minute)
                    day = time_start.day
                    if day < 10:
                        day = '0'+str(day)
                    time_beg = time_start - timedelta(minutes=0.5)
                    time_end = time_start + timedelta(minutes=0.5)
                    sec_beg = time_beg.second
                    sec_end = time_end.second
                    min_beg = time_beg.minute
                    min_end = time_end.minute
                    h_beg = time_beg.hour
                    h_end = time_end.hour
                    d_beg = time_beg.day
                    d_end = time_end.day
                    if sec_beg < 10:
                        sec_beg = '0'+str(sec_beg)
                    if sec_end < 10:
                        sec_end = '0'+str(sec_end)
                    if min_beg < 10:
                        min_beg = '0'+str(min_beg)
                    if min_end < 10:
                        min_end = '0'+str(min_end)
                    if h_beg < 10:
                        h_beg = '0'+str(h_beg)
                    if h_end < 10:
                        h_end = '0'+str(h_end)
                    if d_beg < 10:
                        d_beg = '0'+str(d_beg)
                    if d_end < 10:
                        d_end = '0'+str(d_end)
                    #Add KDP to the dataset
                    #kdp_dict = pyart.retrieve.kdp_proc.kdp_maesaka(radar)
                    print('its this line')
                    #radar.add_field('KDP', kdp_dict[0])
                    print('heres the problem')
                    print(datetime.utcnow())
                    # mask out last 10 gates of each ray, this removes the "ring" around the radar.
                    radar.fields['differential_reflectivity']['data'][:, -10:] = np.ma.masked
                    ref_ungridded = radar.fields['reflectivity']['data']
                    #Mask out everything with reflectivity below Z=20 dBZ for Z and ZDR
                    refl_c = np.copy(ref_ungridded)
                    ref_c = ma.masked_where(refl_c < 20., refl_c)
                    zdr_ungridded = radar.fields['differential_reflectivity']['data']
                    zdrl_c = np.copy(zdr_ungridded)
                    zdr_c = ma.masked_where(refl_c < 20, zdrl_c)
                    #Get ungridded lats/lons
                    #kdp_ungridded = radar.fields['KDP']['data']
                    phidp_ungridded = radar.fields['differential_phase']['data']
                    cc_ungridded = radar.fields['cross_correlation_ratio']['data']
                    print(datetime.utcnow())
                    #NOW add KDP to the dataset
                    #Now calculate KDP manually following NWS methodology
                    #First, get the phidp gradient
                    phidp_gradient = np.asarray(np.gradient(phidp_ungridded))/0.50
                    kdp_raw = phidp_gradient[1,:,:]
                    kdp1raw_c = np.copy(kdp_raw)
                    kdpraw_c = ma.masked_where(kdp_raw > 40., kdp1raw_c)
                    kdp_NWS = np.zeros((kdp_raw.shape[0], kdp_raw.shape[1]))
                    #Do NWS smoothing process
                    for i in range(kdp_raw.shape[0]):
                        for j in range(kdp_raw.shape[1]):
                            if cc_ungridded[i, j] > 0.90:
                                #print(j)
                                if ref_ungridded[i, j] > 40:
                                    try:
                                        kdp_new = np.mean(kdpraw_c[i, j-4:j+4])
                                        kdp_NWS[i, j] = kdp_new
                                        #print(np.mean(kdpraw_c[i, j-4:j+4]))
                                    except:
                                        kdp_NWS[i, j] = 0
                                else:
                                    try:
                                        kdp_new = np.mean(kdpraw_c[i, j-12:j+12])
                                        kdp_NWS[i, j] = kdp_new
                                    except:
                                        kdp_NWS[i, j] = 0  
                    #Mask w/Z
                    kdp1nws_c = np.copy(kdp_NWS)
                    kdpnws_c = ma.masked_where(refl_c < 20., kdp1nws_c)
                    #Create dictionary
                    kdp_nwsdict = {}
                    kdp_nwsdict['units'] = 'degrees/km'
                    kdp_nwsdict['standard_name'] = 'specific_differential_phase_hv'
                    kdp_nwsdict['long_name'] = 'Specific Differential Phase (KDP)'
                    kdp_nwsdict['coordinates'] = 'elevation azimuth range'
                    kdp_nwsdict['data'] = kdpnws_c
                    kdp_nwsdict['valid_min'] = 0.0
                    kdp_nwsdict['Clipf'] = 3906250000.0
                    #Add field to radar
                    radar.add_field('KDP', kdp_nwsdict)
                    #Test data
                    kdp_ungridded_nws = radar.fields['KDP']['data']
                    ungrid_lons = radar.gate_longitude['data']
                    ungrid_lats = radar.gate_latitude['data']
                    print(datetime.utcnow())
                    #Get ungridded gate altitudes
                    gate_altitude = radar.gate_altitude['data'][:]
                    # exclude masked gates from the gridding
                    gatefilter = pyart.filters.GateFilter(radar)
                    gatefilter.exclude_masked('differential_reflectivity')
                    print('almost gridding')
                    #Now let's grid the data on a ~250 m x 250 m grid
                    grid = pyart.map.grid_from_radars(
                        (radar,), gatefilters=(gatefilter, ),
                        grid_shape=(1, 500, 500),
                        grid_limits=((200, 200), (-123000.0, 123000.0), (-123000.0, 123000.0)),
                        fields=['differential_reflectivity','reflectivity','KDP','cross_correlation_ratio'])
                    #Get the data from the grid
                    ZDR = grid.fields['differential_reflectivity']['data'][0]
                    REF = grid.fields['reflectivity']['data'][0]
                    KDP = grid.fields['KDP']['data'][0]
                    CC = grid.fields['cross_correlation_ratio']['data'][0]
                    print(datetime.utcnow())
                    #Mask everything below 20dbz on the grid
                    ZDRmasked1 = ma.masked_where(REF < 20, ZDR)
                    REFmasked = ma.masked_where(REF < 20, REF)
                    #Use a 50 dBZ mask for KDP to only get areas in the storm core. This threshold should be considered more closely
                    KDPmasked = ma.masked_where(REF < 50, KDP)
                    KDPmasked = ma.filled(KDPmasked, fill_value = -2)
                    #Try to filter out spots not in forward flank using Z gradient direction
                    print('made it to smoothing')
                    #First, smooth Z, take the gradient, and find its direction
                    smoothed_ref1 = ndi.gaussian_filter(REFmasked, sigma = 2, order = 0)
                    REFgradient = np.asarray(np.gradient(smoothed_ref1))
                    REFgradient[0,:,:] = ma.masked_where(REF < 20, REFgradient[0,:,:])
                    REFgradient[1,:,:] = ma.masked_where(REF < 20, REFgradient[1,:,:])
                    print('made it through gradient')
                    grad_dir1 = get_wind_dir(REFgradient[1,:,:] * units('m/s'), REFgradient[0,:,:] * units('m/s'))
                    grad_mag = get_wind_speed(REFgradient[1,:,:] * units('m/s'), REFgradient[0,:,:] * units('m/s'))
                    grad_dir = ma.masked_where(REF < 20, grad_dir1)
                    #Get difference between the gradient direction and the FFD gradient direction calculated earlier
                    srdir = storm_relative_dir
                    srirad = np.copy(srdir)*units('degrees').to('radian')
                    grad_dir = grad_dir*units('degrees').to('radian')
                    grad_ffd = np.abs(np.arctan2(np.sin(grad_dir-srirad), np.cos(grad_dir-srirad)))
                    print(grad_ffd)
                    print(np.max(grad_ffd))
                    print(np.min(grad_ffd))
                    grad_ffd = np.asarray(grad_ffd)*units('radian')
                    grad_ex = np.copy(grad_ffd)
                    grad_ffd = grad_ffd.to('degrees')
                    print(grad_ffd)
                    print(np.max(grad_ffd))
                    print(np.min(grad_ffd))
                    #Mask out areas where the difference between the two is too large and the ZDR is likely not in the forward flank
                    ZDRmasked2 = ma.masked_where(grad_ffd > 120 * units('degrees'), ZDRmasked1)
                    ZDRmasked = ma.masked_where(CC < .60, ZDRmasked2)
                    #Add a fill value for the ZDR mask so that contours will be closed
                    ZDRmasked = ma.filled(ZDRmasked, fill_value = -2)
                    #Extract the gridded lats and lons
                    rlons = grid.point_longitude['data']
                    rlats = grid.point_latitude['data']
                    rlons_2d = rlons[0,:,:]
                    rlats_2d = rlats[0,:,:]
                    cenlat = radar.latitude['data'][0]
                    cenlon = radar.longitude['data'][0]
                    #Let's set up the map projection!
                    print('Set up our projection')
                    crs = ccrs.LambertConformal(central_longitude=-100.0, central_latitude=45.0)

                    # Set up our array of latitude and longitude values and transform our data to 
                    # the desired projection.

                    tlatlons = crs.transform_points(ccrs.LambertConformal(central_longitude=265, central_latitude=25, standard_parallels=(25.,25.)),rlons[0,:,:],rlats[0,:,:])
                    tlons = tlatlons[:,:,0]
                    tlats = tlatlons[:,:,1]

                    # Limit the extent of the map area, must convert to proper coords.
                    LL = (cenlon-1.5,cenlat-1.5,ccrs.PlateCarree())
                    UR = (cenlon+1.5,cenlat+1.5,ccrs.PlateCarree())
                    print(LL)

                    # Get data to plot state and province boundaries
                    states_provinces = cfeature.NaturalEarthFeature(
                            category='cultural',
                            name='admin_1_states_provinces_lakes',
                            scale='50m',
                            facecolor='none')
                    #Make sure these shapefiles are in the same directory as the script
                    #Lines with county/state shapefiles are commented out here, so you can add your own
                    #or use the shapefiles provided on the github page
                    #fname = 'cb_2016_us_county_20m/cb_2016_us_county_20m.shp'
                    #fname2 = 'cb_2016_us_state_20m/cb_2016_us_state_20m.shp'
                    #counties = ShapelyFeature(Reader(fname).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
                    #states = ShapelyFeature(Reader(fname2).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
                    #Create a figure and plot up the initial data and contours for the algorithm
                    fig=plt.figure(n,figsize=(30.,25.))
                    ax = plt.subplot(111,projection=ccrs.PlateCarree())
                    ax.coastlines('50m',edgecolor='black',linewidth=0.75)
                    #ax.add_feature(counties, edgecolor = 'black', linewidth = 0.5)
                    #ax.add_feature(states, edgecolor = 'black', linewidth = 1.5)
                    ax.set_extent([LL[0],UR[0],LL[1],UR[1]])
                    REFlevels = np.arange(20,73,2)
                    print('plotting')
                    refp = ax.pcolormesh(ungrid_lons, ungrid_lats, ref_c, cmap=plt.cm.gist_ncar, vmin = 10, vmax = 73)
                    #Homeyer rainbow colormap commented out for now, but can be added back in if you have it
                    #refp = ax.pcolormesh(ungrid_lons, ungrid_lats, ref_c, cmap='HomeyerRainbow', vmin = 10, vmax = 73)
                    #Option to have a ZDR background instead of Z:
                    #zdrp = ax.pcolormesh(ungrid_lons, ungrid_lats, zdr_c, cmap=plt.cm.nipy_spectral, vmin = -2, vmax = 6)
                    ##
                    #Storm tracking algorithm starts here
                    ##
                    #Reflectivity smoothed for storm tracker
                    smoothed_ref = ndi.gaussian_filter(REFmasked, sigma = 3, order = 0)
                    #REFlev = [45]
                    #REFlev1 = [50]
                    #1st Z contour plotted
                    refc = ax.contour(rlons[0,:,:],rlats[0,:,:],smoothed_ref,REFlev, alpha=.4)
                    #Empty arrays made for storm characteristics
                    ref_areas = []
                    max_lons_c = []
                    max_lats_c = []
                    storm_ids = []
                    #Set up projection for area calculations
                    proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                               pyproj.Proj(init='epsg:3857'))

                    #Main part of storm tracking algorithm starts by looping through all contours looking for Z centroids
                    #This method for breaking contours into polygons based on this stack overflow tutorial:
                    #https://gis.stackexchange.com/questions/99917/converting-matplotlib-contour-objects-to-shapely-objects
                    for level in refc.collections:
                        #Loops through each closed polygon in the contour 
                        for contour_poly in level.get_paths(): 
                            for n_contour,contour in enumerate(contour_poly.to_polygons()):
                                print(1)
                                contour_a = np.asarray(contour[:])
                                xa = contour_a[:,0]
                                ya = contour_a[:,1]
                                polygon_new = geometry.Polygon([(i[0], i[1]) for i in zip(xa,ya)])
                                #Eliminates 'holes' in the polygons
                                if n_contour == 0:
                                    polygon = polygon_new
                                else:
                                    polygon = polygon.difference(polygon_new)

                            print(polygon.centroid.x)
                            #Transform the polygon's coordinates to the proper projection and calculate area
                            pr_area = (transform(proj, polygon).area * units('m^2')).to('km^2')
                            #Use the polygon boundary to select all points within the polygon via a mask
                            boundary = np.asarray(polygon.boundary.xy)
                            polypath = Path(boundary.transpose())
                            coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T 
                            maskr = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                            meanr = np.mean(smoothed_ref[maskr])
                            print('past mask')
                            print(meanr)
                            if pr_area > 10 * units('km^2') and meanr > REFlev[0]:
                                print('found a storm')
                                #For big blobs with embedded supercells, find the embedded storm cores
                                #Normal 'big storm' cutoff 300 km^2
                                if pr_area > big_storm * units('km^2'):
                                    print('found a big storm')
                                    rlon_2 = rlons[0,:,:]
                                    rlat_2 = rlats[0,:,:]
                                    #smoothed_ref_m = ma.MaskedArray(smoothed_ref, mask=maskr)
                                    smoothed_ref_m = ma.masked_where(maskr==False, smoothed_ref)
                                    smoothed_ref_m = ma.filled(smoothed_ref_m, fill_value = -2)
                                    rlon2m = ma.MaskedArray(rlon_2, mask=maskr)
                                    rlat2m = ma.MaskedArray(rlat_2, mask=maskr)
                                    #This section uses the 2nd reflectivity threshold to subdivide big storms, in a similar manner to the 
                                    #previous section's method for finding storms
                                    refc1 = ax.contour(rlon2m,rlat2m,smoothed_ref_m,REFlev1, linewidths = 3, linestyle = '--', alpha=.4)
                                    #refc1 = ax.contour(rlon_2[maskr],rlat_2[maskr],smoothed_ref[maskr],REFlev1, colors = 'g', linewidths = 3)
                                    print('plotted a big storm')
                                    #Look for reflectivity centroids
                                    for level1 in refc1.collections:
                                        print('made it to beginning of loop')
                                        for contour_poly1 in level1.get_paths(): 
                                            for n_contour1,contour1 in enumerate(contour_poly1.to_polygons()):
                                                print(2)
                                                contour_a1 = np.asarray(contour1[:])
                                                xa1 = contour_a1[:,0]
                                                ya1 = contour_a1[:,1]
                                                polygon_new1 = geometry.Polygon([(i[0], i[1]) for i in zip(xa1,ya1)])
                                                if n_contour1 == 0:
                                                    polygon1 = polygon_new1
                                                else:
                                                    polygon1 = polygon1.difference(polygon_new1)

                                            print(polygon1.centroid.x)
                                            pr_area1 = (transform(proj, polygon1).area * units('m^2')).to('km^2')
                                            boundary1 = np.asarray(polygon1.boundary.xy)
                                            polypath1 = Path(boundary1.transpose())
                                            maskr1 = polypath1.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                                            meanr1 = np.mean(smoothed_ref[maskr1])
                                            #Add objects that fit requirements to the list of storm objects
                                            if pr_area1 > 10 * units('km^2') and meanr1 > REFlev1[0]:
                                                ref_areas.append((pr_area1.magnitude*2))
                                                max_lons_c.append((polygon1.centroid.x))
                                                max_lats_c.append((polygon1.centroid.y))
                                                #For tracking, assign ID numbers and match current storms to any previous storms that are close enough to 
                                                #be the same
                                                if scan_index == 0:
                                                    storm_ids.append((storm_index))
                                                    storm_index = storm_index + 1
                                                else:
                                                    #dist_track = np.zeros((np.asarray(max_lons_p).shape[0]))
                                                    max_lons_p = np.asarray(tracks_dataframe['storm_lon'].loc[scan_index-1].iloc[:])
                                                    max_lats_p = np.asarray(tracks_dataframe['storm_lat'].loc[scan_index-1].iloc[:])
                                                    storm_ids_p = np.asarray(tracks_dataframe['storm_id1'].loc[scan_index-1].iloc[:])
                                                    dist_track = np.zeros((np.asarray(max_lons_p).shape[0]))
                                                    for i in range(max_lons_p.shape[0]):
                                                        distance_track = g.inv(polygon1.centroid.x, polygon1.centroid.y,
                                                                               max_lons_p[i], max_lats_p[i])
                                                        dist_track[i] = distance_track[2]/1000.
                                                    print(dist_track)
                                                    print('Poly lon', polygon1.centroid.x)
                                                    print(max_lons_p)
                                                    print(storm_ids_p)
                                                    if np.min(dist_track) < 10.0:
                                                        storm_ids.append((storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]]))
                                                        print('storm id', storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]])
                                                    else:
                                                        storm_ids.append((storm_index))
                                                        print('storm id', storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]])
                                                        storm_index = storm_index + 1
                                                print('added polygon')
                                            else:
                                                print('nope')
                                #Do the same thing for objects from the 1st reflectivity threshold
                                else:
                                    ref_areas.append((pr_area.magnitude))
                                    max_lons_c.append((polygon.centroid.x))
                                    max_lats_c.append((polygon.centroid.y))
                                    if scan_index == 0:
                                        storm_ids.append((storm_index))
                                        storm_index = storm_index + 1
                                    else:
                                        #dist_track = np.zeros((np.asarray(max_lons_p).shape[0]))
                                        max_lons_p = np.asarray(tracks_dataframe['storm_lon'].loc[scan_index-1].iloc[:])
                                        max_lats_p = np.asarray(tracks_dataframe['storm_lat'].loc[scan_index-1].iloc[:])
                                        storm_ids_p = np.asarray(tracks_dataframe['storm_id1'].loc[scan_index-1].iloc[:])
                                        dist_track = np.zeros((np.asarray(max_lons_p).shape[0]))
                                        for i in range(max_lons_p.shape[0]):
                                            distance_track = g.inv(polygon.centroid.x, polygon.centroid.y,
                                                                   max_lons_p[i], max_lats_p[i])
                                            dist_track[i] = distance_track[2]/1000.
                                        print(dist_track)
                                        print('Poly lon', polygon.centroid.x)
                                        print(max_lons_p)
                                        print(storm_ids_p)
                                        if np.min(dist_track) < 10.0:
                                            storm_ids.append((storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]]))
                                            print('storm id', storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]])
                                        else:
                                            storm_ids.append((storm_index))
                                            print('storm id', storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]])
                                            storm_index = storm_index + 1
                                    print('added polygon')

                        #print(s_new)
                    #Setup tracking index for storm of interest
                    tracking_ind=np.where(np.asarray(storm_ids)==storm_to_track)[0]
                    print('tracking id')
                    print(tracking_ind)
                    max_lons_c = np.asarray(max_lons_c)
                    max_lats_c = np.asarray(max_lats_c)
                    ref_areas = np.asarray(ref_areas)
                    #Create the ZDR and KDP contours which will later be broken into polygons
                    zdrc = ax.contour(rlons[0,:,:],rlats[0,:,:],ZDRmasked,zdrlev,linewidths = 2, colors='purple', alpha = .7)
                    kdpc = ax.contour(rlons[0,:,:],rlats[0,:,:],KDPmasked,kdplev,linewidths = 2, colors='green', alpha = .8)

                    print('made it here')
                    plt.savefig('testfig.png')

                    #Create ZDR arc objects using a similar method as employed in making the storm objects
                    if len(max_lons_c) > 0:
                        zdr_areas = []
                        zdr_centroid_lon = []
                        zdr_centroid_lat = []
                        zdr_mean = []
                        zdr_cc_mean = []
                        zdr_max = []
                        zdr_storm_lon = []
                        zdr_storm_lat = []
                        zdr_dist = []
                        zdr_forw = []
                        zdr_back = []
                        zdr_masks = []
                        #print("here too")
                        #Break contours into polygons using the same method as for reflectivity
                        for level in zdrc.collections:
                            for contour_poly in level.get_paths(): 
                                for n_contour,contour in enumerate(contour_poly.to_polygons()):
                                    #print('hi')
                                    contour_a = np.asarray(contour[:])
                                    xa = contour_a[:,0]
                                    ya = contour_a[:,1]
                                    polygon_new = geometry.Polygon([(i[0], i[1]) for i in zip(xa,ya)])
                                    if n_contour == 0:
                                        polygon = polygon_new
                                        #print('hi')
                                    else:
                                        polygon = polygon.difference(polygon_new)
                                        #print('hi')
                                pr_area = (transform(proj, polygon).area * units('m^2')).to('km^2')
                                boundary = np.asarray(polygon.boundary.xy)
                                polypath = Path(boundary.transpose())
                                coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T 
                                mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                                mean = np.mean(ZDRmasked[mask])
                                mean_cc = np.mean(CC[mask])
                                mean_Z = np.mean(REF[mask])
                                mean_graddir = np.mean(grad_ffd[mask])
                                mean_grad = np.mean(grad_mag[mask])
                                mean_kdp = np.mean(KDP[mask])
                                #Only select out objects larger than 1 km^2 with high enough CC
                                if pr_area > 1 * units('km^2') and mean > zdrlev[0] and mean_cc > .88:
                                    g = Geod(ellps='sphere')
                                    dist = np.zeros((np.asarray(max_lons_c).shape[0]))
                                    forw = np.zeros((np.asarray(max_lons_c).shape[0]))
                                    rawangle = np.zeros((np.asarray(max_lons_c).shape[0]))
                                    back = np.zeros((np.asarray(max_lons_c).shape[0]))
                                    zdr_polypath = polypath
                                    #Assign ZDR arc objects to the nearest acceptable storm object
                                    for i in range(dist.shape[0]):
                                                distance_1 = g.inv(polygon.centroid.x, polygon.centroid.y,
                                                                       max_lons_c[i], max_lats_c[i])
                                                #print(distance_1[2]/1000)
                                                #print(distance_1)
                                                back[i] = distance_1[1]
                                                #print('Raw back angle', back[i])
                                                if distance_1[1] < 0:
                                                    back[i] = distance_1[1] + 360
                                                #print('fixed back', back[i])
                                                forw[i] = np.abs(back[i] - storm_relative_dir)
                                                #print('raw forw', forw[i])
                                                rawangle[i] = back[i] - storm_relative_dir
                                                #Account for weird angles
                                                if forw[i] > 180:
                                                    #print('Big angle')
                                                    forw[i] = 360 - forw[i]
                                                    rawangle[i] = (360-forw[i])*(-1)
                                                    #print(rawangle[i])
                                                dist[i] = distance_1[2]/1000.
                                                rawangle[i] = rawangle[i]*(-1)
                                                #print('fixed forw', forw[i])
                                                #print('raw angle', rawangle[i])
                                    #print(dist.shape)
                                    #Set search radii around storms for ZDR arcs objects
                                    #Outer Radius-variable, 5km past the square root of the storm object's area
                                    #Outer_r = (1.0*np.sqrt(ref_areas[np.where(dist == np.min(dist))[0][0]]))
                                    #Outer_r = 15.0
                                    #Inner Radius-variable, 2km pask the square root of 1/4 of the storm's area
                                    #Inner_r = (0.25*np.sqrt(ref_areas[np.where(dist == np.min(dist))[0][0]]))
                                    #Inner_r = 6.0
                                    #Pick out only ZDR arc objects with a reasonable probability of actually being in the FFD region
                                    #using their location relative to the storm centroid
                                    if (forw[np.where(dist == np.min(dist))[0][0]] < 180 and np.min(dist) < Outer_r) or (forw[np.where(dist == np.min(dist))[0][0]] < 140 and np.min(dist) < Inner_r):
                                        #Use ML algorithm to eliminate non-arc objects
                                        #Get x and y components
                                        if (rawangle[np.where(dist == np.min(dist))[0][0]] > 0):
                                            directions_raw = 360 - rawangle[np.where(dist == np.min(dist))[0][0]]
                                        else:
                                            directions_raw = (-1) * rawangle[np.where(dist == np.min(dist))[0][0]]
                                        
                                        xc, yc = get_wind_components(np.min(dist), directions_raw * units('degree'))
                                        print('got xc')
                                        ARC_X = np.zeros((1, 12))
                                        print('got array1')
                                        ARC_X[:,0] = pr_area.magnitude
                                        print('got array2')
                                        ARC_X[:,1] = np.min(dist)
                                        print('got array3')
                                        ARC_X[:,2] = np.max(ZDRmasked[mask]) / mean
                                        print('got array4')
                                        ARC_X[:,3] = (np.max(ZDRmasked[mask]) / mean) * pr_area.magnitude
                                        print('got array5')
                                        ARC_X[:,4] = mean_cc
                                        print('got array6')
                                        ARC_X[:,5] = mean_kdp
                                        print('got array7')
                                        ARC_X[:,6] = mean_Z
                                        print('got array8')
                                        ARC_X[:,7] = mean_graddir
                                        print('got array9')
                                        ARC_X[:,8] = mean_grad
                                        print('got array10')
                                        ARC_X[:,9] = rawangle[np.where(dist == np.min(dist))[0][0]]
                                        print('got array11')
                                        ARC_X[:,10] = xc
                                        print('got array12')
                                        ARC_X[:,11] = yc
                                        print('got to prediction')
                                        pred_zdr = forest_loaded.predict(ARC_X)
                                        print(pred_zdr)
                                        if (pred_zdr[0]==1):
                                            print('arc')
                                            zdr_storm_lon.append((max_lons_c[np.where(dist == np.min(dist))[0][0]]))
                                            zdr_storm_lat.append((max_lats_c[np.where(dist == np.min(dist))[0][0]]))
                                            zdr_dist.append(np.min(dist))
                                            zdr_forw.append(forw[np.where(dist == np.min(dist))[0][0]])
                                            zdr_back.append(back[np.where(dist == np.min(dist))[0][0]])
                                            zdr_areas.append((pr_area))
                                            zdr_centroid_lon.append((polygon.centroid.x))
                                            zdr_centroid_lat.append((polygon.centroid.y))
                                            zdr_mean.append((mean))
                                            zdr_cc_mean.append((mean_cc))
                                            zdr_max.append((np.max(ZDRmasked[mask])))
                                            zdr_masks.append(mask)
                                            patch = PathPatch(polypath, facecolor='none', alpha=.8, edgecolor = 'blue', linewidth = 3)
                                            ax.add_patch(patch)
                                            #Add polygon to placefile
                                            f.write('TimeRange: '+str(time_start.year)+'-'+str(month)+'-'+str(d_beg)+'T'+str(h_beg)+':'+str(min_beg)+':'+str(sec_beg)+'Z '+str(time_start.year)+'-'+str(month)+'-'+str(d_end)+'T'+str(h_end)+':'+str(min_end)+':'+str(sec_end)+'Z')
                                            f.write('\n')
                                            f.write("Color: 000 000 139 \n")
                                            f.write('Line: 3, 0, "ZDR Arc Outline" \n')
                                            for i in range(len(zdr_polypath.vertices)):
                                                f.write("%.5f" %(zdr_polypath.vertices[i][1]))
                                                f.write(", ")
                                                f.write("%.5f" %(zdr_polypath.vertices[i][0]))
                                                f.write('\n')
                                            #f.write(str(zdr_polypath.vertices[0][0])+', '+str(zdr_polypath.vertices[0][1])+'\n')
                                            f.write("End: \n \n")
                                        #if (((max_lons_c[np.where(dist == np.min(dist))[0][0]]) in max_lons_c[tracking_ind]) and ((max_lats_c[np.where(dist == np.min(dist))[0][0]]) in max_lats_c[tracking_ind])):
                                        #    plt.text(float(polygon.centroid.x), float(polygon.centroid.y), "%.1f" %(float(object_number)), size = 23, color = 'purple')
                                            #Add a line that writes all of the attibutes of each object to a csv
                                            #Tornadic filename
                                            #with open('Machine_Learning/ML_test'+station+str(dt.year)+str(dt.month)+str(dt.day)+str(dt.hour)+str(dt.minute)+'.csv', 'a') as csvfile:
                                            #Nontornadic filename
                                           # with open('Machine_Learning/NT2_ML_test'+station+str(dt.year)+str(dt.month)+str(dt.day)+str(dt.hour)+str(dt.minute)+'.csv', 'a') as csvfile:
                                            #    fieldnames = ['number', 'hour', 'minute','area','distance','angle','mean','max','mean_cc','mean_kdp','mean_Z','mean_graddir','mean_grad', 'raw_angle']
                                            #    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                                            #    writer.writerow({'number': object_number, 'hour': hour, 'minute': minute, 'area': pr_area.magnitude, 'distance': np.min(dist), 'angle': forw[np.where(dist == np.min(dist))[0][0]], 'mean': mean, 'max': np.max(ZDRmasked[mask]), 'mean_cc': mean_cc, 'mean_kdp': mean_kdp, 'mean_Z': mean_Z, 'mean_graddir': mean_graddir.magnitude, 'mean_grad': mean_grad.magnitude, 'raw_angle': rawangle[np.where(dist == np.min(dist))[0][0]]})
                                            #object_number=object_number+1
                                                     #print(s_new)
                        print('made it through zdr centroids')
                        #Identify KDP foot objects in a similar way to the ZDR arc objects
                        if len(max_lons_c) > 0:
                            kdp_areas = []
                            kdp_centroid_lon = []
                            kdp_centroid_lat = []
                            kdp_max = []
                            kdp_storm_lon = []
                            kdp_storm_lat = []
                            for level in kdpc.collections:
                                for contour_poly in level.get_paths(): 
                                    for n_contour,contour in enumerate(contour_poly.to_polygons()):
                                        print(1)
                                        contour_a = np.asarray(contour[:])
                                        xa = contour_a[:,0]
                                        ya = contour_a[:,1]
                                        polygon_new = geometry.Polygon([(i[0], i[1]) for i in zip(xa,ya)])
                                        if n_contour == 0:
                                            polygon = polygon_new
                                        else:
                                            polygon = polygon.difference(polygon_new)

                                    print(polygon.centroid.x)
                                    pr_area = (transform(proj, polygon).area * units('m^2')).to('km^2')
                                    boundary = np.asarray(polygon.boundary.xy)
                                    polypath = Path(boundary.transpose())
                                    coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                                    mask_kdp = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                                    #mean = np.mean(ZDRmasked[mask])
                                    #mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                                    #mean = np.mean(REFmasked[mask])
                                    if pr_area > 2 * units('km^2'):
                                        g = Geod(ellps='sphere')
                                        dist_kdp = np.zeros((np.asarray(max_lons_c).shape[0]))
                                        for i in range(dist_kdp.shape[0]):
                                                    distance_kdp = g.inv(polygon.centroid.x, polygon.centroid.y,
                                                                           max_lons_c[i], max_lats_c[i])
                                                    #print(distance_1[2]/1000)
                                                    #print("KDP dist:", distance_kdp)
                                                    dist_kdp[i] = distance_kdp[2]/1000.
                                        print(dist_kdp)
                                        if np.min(np.asarray(dist_kdp)) < 15.0:
                                            #print('Got to KDP stuff')
                                            kdp_path = polypath
                                            kdp_areas.append((pr_area))
                                            kdp_centroid_lon.append((polygon.centroid.x))
                                            kdp_centroid_lat.append((polygon.centroid.y))
                                            kdp_storm_lon.append((max_lons_c[np.where(dist_kdp == np.min(dist_kdp))[0][0]]))
                                            kdp_storm_lat.append((max_lats_c[np.where(dist_kdp == np.min(dist_kdp))[0][0]]))
                                            kdp_max.append((np.max(KDPmasked[mask_kdp])))
                                            patch = PathPatch(polypath, facecolor='none', alpha=.5, edgecolor = 'grey', linewidth = 3)
                                            ax.add_patch(patch)
                                            #Add polygon to placefile
                                            #f.write('TimeRange: '+str(time_start.year)+'-'+str(month)+'-'+str(d_beg)+'T'+str(h_beg)+':'+str(min_beg)+':00Z '+str(time_start.year)+'-'+str(month)+'-'+str(d_end)+'T'+str(h_end)+':'+str(min_end)+':00Z')
                                            f.write('TimeRange: '+str(time_start.year)+'-'+str(month)+'-'+str(d_beg)+'T'+str(h_beg)+':'+str(min_beg)+':'+str(sec_beg)+'Z '+str(time_start.year)+'-'+str(month)+'-'+str(d_end)+'T'+str(h_end)+':'+str(min_end)+':'+str(sec_end)+'Z')
                                            f.write('\n')
                                            f.write("Color: 000 139 000 \n")
                                            f.write('Line: 3, 0, "KDP Foot Outline" \n')
                                            for i in range(len(kdp_path.vertices)):
                                                f.write("%.5f" %(kdp_path.vertices[i][1]))
                                                f.write(", ")
                                                f.write("%.5f" %(kdp_path.vertices[i][0]))
                                                f.write('\n')
                                            #f.write(str(zdr_polypath.vertices[0][0])+', '+str(zdr_polypath.vertices[0][1])+'\n')
                                            f.write("End: \n \n")

                            print('made it through kdp centroids')

                            #Consolidating the arc objects associated with each storm:
                            zdr_areas_arr = np.zeros((len(zdr_areas)))
                            zdr_max_arr = np.zeros((len(zdr_max)))
                            zdr_mean_arr = np.zeros((len(zdr_mean)))                    
                            for i in range(len(zdr_areas)):
                                zdr_areas_arr[i] = zdr_areas[i].magnitude
                                zdr_max_arr[i] = zdr_max[i]
                                zdr_mean_arr[i] = zdr_mean[i]

                            zdr_centroid_lons = np.asarray(zdr_centroid_lon)
                            zdr_centroid_lats = np.asarray(zdr_centroid_lat)
                            zdr_con_areas = []
                            zdr_con_maxes = []
                            zdr_con_means = []
                            zdr_con_centroid_lon = []
                            zdr_con_centroid_lat = []
                            zdr_con_max_lon = []
                            zdr_con_max_lat = []
                            zdr_con_storm_lon = []
                            zdr_con_storm_lat = []
                            zdr_con_masks = []
                            zdr_con_dev = []
                            zdr_con_10max = []
                            zdr_con_mode = []
                            zdr_con_median = []
                            zdr_masks = np.asarray(zdr_masks)
                            #Consolidate KDP objects as well
                            kdp_areas_arr = np.zeros((len(kdp_areas)))
                            kdp_max_arr = np.zeros((len(kdp_max)))
                            for i in range(len(kdp_areas)):
                                kdp_areas_arr[i] = kdp_areas[i].magnitude
                                kdp_max_arr[i] = kdp_max[i]
                            kdp_centroid_lons = np.asarray(kdp_centroid_lon)
                            kdp_centroid_lats = np.asarray(kdp_centroid_lat)
                            kdp_con_areas = []
                            kdp_con_maxes = []
                            kdp_con_centroid_lon = []
                            kdp_con_centroid_lat = []
                            kdp_con_max_lon = []
                            kdp_con_max_lat = []
                            kdp_con_storm_lon = []
                            kdp_con_storm_lat = []
                            for i in enumerate(zdr_storm_lon):
                                print(i[0])
                                if i[0] != 0:
                                    if zdr_storm_lon[i[0]-1] == zdr_storm_lon[i[0]]:
                                        #print("Skipping this one")
                                        continue
                                    else:
                                        print(zdr_storm_lon[i[0]])
                                        #Find the arc objects associated with this storm:
                                        zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                        zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                        #print("zdr lons:", zdr_objects_lons)
                                        #Get the sum of their areas
                                        print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
                                        zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                        zdr_con_maxes.append(np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                        zdr_con_means.append(np.mean(zdr_mean_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                        zdr_con_max_lon.append(rlons_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                                        zdr_con_max_lat.append(rlats_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                                        #print("Areas sum:", zdr_con_areas)
                                        #Find the actual centroids
                                        weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                        zdr_con_centroid_lon.append(np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                        weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                        zdr_con_centroid_lat.append(np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                        zdr_con_storm_lon.append(zdr_storm_lon[i[0]])
                                        zdr_con_storm_lat.append(zdr_storm_lat[i[0]])
                                        zdr_con_masks.append(np.sum(zdr_masks[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])],axis=0, dtype=bool))
                                        mask_con = np.sum(zdr_masks[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])], axis=0, dtype=bool)
                                        zdr_con_dev.append(np.std(ZDRmasked[mask_con]))
                                        ZDRsorted = np.sort(ZDRmasked[mask_con])[::-1]
                                        zdr_con_10max.append(np.mean(ZDRsorted[0:10]))
                                        zdr_con_mode.append(stats.mode(ZDRmasked[mask_con]))
                                        zdr_con_median.append(np.median(ZDRmasked[mask_con]))
                                        #print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
                                        #print("lons in loop", zdr_objects_lons)

                                        try:
                                            #Find the kdp objects associated with this storm:
                                            kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                            #print("kdp lons:", kdp_objects_lons)
                                            kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                            #Get the sum of their areas
                                            print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                            kdp_con_areas.append(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                            kdp_con_maxes.append(np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                            kdp_con_max_lon.append(rlons_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                            kdp_con_max_lat.append(rlats_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                            #Find the actual centroids
                                            weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                            kdp_con_centroid_lon.append(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                            weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                            #print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                            kdp_con_centroid_lat.append(np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                            kdp_con_storm_lon.append(zdr_storm_lon[i[0]])
                                            kdp_con_storm_lat.append(zdr_storm_lat[i[0]])
                                        except:
                                            print('storm missing kdp or zdr')
                                            kdp_con_areas.append(0)
                                            kdp_con_maxes.append(0)
                                            kdp_con_max_lon.append(0)
                                            kdp_con_max_lat.append(0)
                                            kdp_con_centroid_lon.append(0)
                                            kdp_con_centroid_lat.append(0)
                                            kdp_con_storm_lon.append(0)
                                            kdp_con_storm_lat.append(0)



                                else:
                                    #print(zdr_storm_lon[i[0]])
                                    #Find the arc objects associated with this storm:
                                    zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                    zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                    #print("zdr lons:", zdr_objects_lons)
                                    #print("arc lats:", zdr_objects_lats)
                                    #Get the sum of their areas
                                    #print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
                                    zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                    zdr_con_maxes.append(np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                    zdr_con_means.append(np.mean(zdr_mean_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                    zdr_con_max_lon.append(rlons_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                                    zdr_con_max_lat.append(rlats_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                                    #print("Areas sum:",zdr_con_areas)
                                    #Find the actual centroids
                                    weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                    zdr_con_centroid_lon.append(np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                    weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                    zdr_con_centroid_lat.append(np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                    zdr_con_storm_lon.append(zdr_storm_lon[i[0]])
                                    zdr_con_storm_lat.append(zdr_storm_lat[i[0]])
                                    zdr_con_masks.append(np.sum(zdr_masks[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])], axis=0, dtype=bool))
                                    mask_con = np.sum(zdr_masks[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])], axis=0, dtype=bool)
                                    zdr_con_dev.append(np.std(ZDRmasked[mask_con]))
                                    ZDRsorted = np.sort(ZDRmasked[mask_con])[::-1]
                                    zdr_con_10max.append(np.mean(ZDRsorted[0:10]))
                                    zdr_con_mode.append(stats.mode(ZDRmasked[mask_con]))
                                    zdr_con_median.append(np.median(ZDRmasked[mask_con]))
                                    #print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
                                    #print("lons out of loop", zdr_objects_lons)
                                    try:
                                        #Find the kdp objects associated with this storm:
                                        kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                        #print("kdp lons:", kdp_objects_lons)
                                        kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                        #Get the sum of their areas
                                        #print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                        kdp_con_areas.append(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                        kdp_con_maxes.append(np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                        kdp_con_max_lon.append(rlons_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                        kdp_con_max_lat.append(rlats_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                        #Find the actual centroids
                                        weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                        kdp_con_centroid_lon.append(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                        weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                        #print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                        kdp_con_centroid_lat.append(np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                        kdp_con_storm_lon.append(zdr_storm_lon[i[0]])
                                        kdp_con_storm_lat.append(zdr_storm_lat[i[0]])
                                    except:
                                        print('storm missing kdp or zdr')
                                        kdp_con_areas.append(0)
                                        kdp_con_maxes.append(0)
                                        kdp_con_max_lon.append(0)
                                        kdp_con_max_lat.append(0)
                                        kdp_con_centroid_lon.append(0)
                                        kdp_con_centroid_lat.append(0)
                                        kdp_con_storm_lon.append(0)
                                        kdp_con_storm_lat.append(0)

                            #Calculate KDP-ZDR separation
                            print('calculating separation')
                            kdp_con_centroid_lons1 = np.asarray(kdp_con_centroid_lon)
                            kdp_con_centroid_lats1 = np.asarray(kdp_con_centroid_lat)
                            zdr_con_centroid_lons1 = np.asarray(zdr_con_centroid_lon)
                            zdr_con_centroid_lats1 = np.asarray(zdr_con_centroid_lat)
                            #Eliminate consolidated arcs smaller than a specified area
                            area = 2 #km*2
                            zdr_con_areas_arr = np.asarray(zdr_con_areas)
                            zdr_con_centroid_lats = zdr_con_centroid_lats1[zdr_con_areas_arr > area]
                            zdr_con_centroid_lons = zdr_con_centroid_lons1[zdr_con_areas_arr > area]
                            kdp_con_centroid_lats = kdp_con_centroid_lats1[zdr_con_areas_arr > area]
                            kdp_con_centroid_lons = kdp_con_centroid_lons1[zdr_con_areas_arr > area]
                            zdr_con_max_lons1 = np.asarray(zdr_con_max_lon)[zdr_con_areas_arr > area]
                            zdr_con_max_lats1 = np.asarray(zdr_con_max_lat)[zdr_con_areas_arr > area]
                            kdp_con_max_lons1 = np.asarray(kdp_con_max_lon)[zdr_con_areas_arr > area]
                            kdp_con_max_lats1 = np.asarray(kdp_con_max_lat)[zdr_con_areas_arr > area]
                            print('Boolean problem here')
                            zdr_con_areas1 = zdr_con_areas_arr[zdr_con_areas_arr > area]

                            kdp_inds = np.where(kdp_con_centroid_lats > 0)
                            distance_kdp_zdr = g.inv(kdp_con_centroid_lons[kdp_inds], kdp_con_centroid_lats[kdp_inds], zdr_con_centroid_lons[kdp_inds], zdr_con_centroid_lats[kdp_inds])
                            dist_kdp_zdr = distance_kdp_zdr[2] / 1000.
                            #Now make an array for the distances which will have the same shape as the lats to prevent errors
                            shaped_dist = np.zeros((np.shape(zdr_con_areas)))
                            shaped_dist[kdp_inds] = dist_kdp_zdr
                            print('maybe its here')
                            #Do the same for the distances between the maxes
                            distance_kdp_zdr_max = g.inv(kdp_con_max_lons1[kdp_inds], kdp_con_max_lats1[kdp_inds], zdr_con_max_lons1[kdp_inds], zdr_con_max_lats1[kdp_inds])
                            dist_kdp_zdr_max = distance_kdp_zdr_max[2] / 1000.
                            #Now make an array for the distances which will have the same shape as the lats to prevent errors
                            shaped_dist_max = np.zeros((np.shape(zdr_con_areas)))
                            shaped_dist_max[kdp_inds] = dist_kdp_zdr_max
                            print('or not')
                            print('made it to angles')
                            #Get separation angle for KDP-ZDR centroids
                            back_k = distance_kdp_zdr[1]
                            #print('Raw back angle', back[i])
                            for i in range(back_k.shape[0]):
                                print('loop is ok')
                                if distance_kdp_zdr[1][i] < 0:
                                    print('if is ok')
                                    back_k[i] = distance_kdp_zdr[1][i] + 360
                            print('through loop 1')
                            #print('fixed back', back[i])
                            forw_k = np.abs(back_k - storm_relative_dir)
                            #print('raw forw', forw[i])
                            rawangle_k = back_k - storm_relative_dir
                            #Account for weird angles
                            for i in range(back_k.shape[0]):
                                if forw_k[i] > 180:
                                    #print('Big angle')
                                    forw_k[i] = 360 - forw_k[i]
                                    rawangle_k[i] = (360-forw_k[i])*(-1)
                                #print(rawangle[i])
                            print('through loop 2')
                            rawangle_k = rawangle_k*(-1)
                            
                            #Now make an array for the distances which will have the same shape as the lats to prevent errors
                            shaped_ang = np.zeros((np.shape(zdr_con_areas)))
                            shaped_ang[kdp_inds] = rawangle_k
                            shaped_ang = (180-np.abs(shaped_ang))*(shaped_ang/np.abs(shaped_ang))

                        else:
                            print('No ZDR arcs')
                            kdp_areas = []
                            kdp_centroid_lon = []
                            kdp_centroid_lat = []
                            kdp_storm_lon = []
                            kdp_storm_lat = []
                            zdr_con_centroid_lats = []
                            zdr_con_centroid_lons = []
                            kdp_con_centroid_lats = []
                            kdp_con_centroid_lons = []
                            kdp_con_area = []
                            zdr_con_areas1 = []

                        ###Now let's consolidate everything to fit the Pandas dataframe!
                        p_zdr_areas = []
                        p_zdr_maxes = []
                        p_zdr_means = []
                        p_zdr_devs = []
                        p_zdr_10max = []
                        p_zdr_mode = []
                        p_zdr_median = []
                        p_separations = []
                        p_sp_angle = []
                        if len(zdr_storm_lon) > 0:
                            for storm in enumerate(max_lons_c):
                                print(storm)
                                print(np.flatnonzero(np.isclose(max_lons_c[storm[0]], zdr_con_storm_lon, rtol=1e-05)))
                                matching_ind = np.flatnonzero(np.isclose(max_lons_c[storm[0]], zdr_con_storm_lon, rtol=1e-05))
                                if matching_ind.shape[0] > 0:
                                    p_zdr_areas.append((zdr_con_areas[matching_ind[0]]))
                                    p_zdr_maxes.append((zdr_con_maxes[matching_ind[0]]))
                                    p_zdr_means.append((zdr_con_means[matching_ind[0]]))
                                    p_zdr_devs.append((zdr_con_dev[matching_ind[0]]))
                                    p_zdr_10max.append((zdr_con_10max[matching_ind[0]]))
                                    p_zdr_mode.append((zdr_con_mode[matching_ind[0]]))
                                    p_zdr_median.append((zdr_con_median[matching_ind[0]]))
                                    p_separations.append((shaped_dist[matching_ind[0]]))
                                    p_sp_angle.append((shaped_ang[matching_ind[0]]))
                                else:
                                    p_zdr_areas.append((0))
                                    p_zdr_maxes.append((0))
                                    p_zdr_means.append((0))
                                    p_zdr_devs.append((0))
                                    p_zdr_10max.append((0))
                                    p_zdr_mode.append((0))
                                    p_zdr_median.append((0))
                                    p_separations.append((0))
                                    p_sp_angle.append((0))
                        else:
                            for storm in enumerate(max_lons_c):
                                p_zdr_areas.append((0))
                                p_zdr_maxes.append((0))
                                p_zdr_means.append((0))
                                p_zdr_devs.append((0))
                                p_zdr_10max.append((0))
                                p_zdr_mode.append((0))
                                p_zdr_median.append((0))
                                p_separations.append((0))
                                p_sp_angle.append((0))

                        #Now start plotting stuff!
                        print('made it through giant if statement')
                        if np.asarray(zdr_centroid_lon).shape[0] > 0:
                            ax.scatter(zdr_centroid_lon, zdr_centroid_lat, marker = '*', s = 100, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
                            #ax.scatter(zdr_con_max_lon, zdr_con_max_lat, marker = '*', s = 100, color = 'purple', zorder = 10, transform=ccrs.PlateCarree())
                        if np.asarray(kdp_centroid_lon).shape[0] > 0:
                            ax.scatter(kdp_centroid_lon, kdp_centroid_lat, marker = '^', s = 100, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
                            #ax.scatter(kdp_con_max_lon, kdp_con_max_lat, marker = '^', s = 100, color = 'purple', zorder = 10, transform=ccrs.PlateCarree())
                        print("plotted centroids")
                        #Uncomment to print all object areas
                        #for i in enumerate(zdr_areas):
                        #    plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_areas[i[0]].magnitude), size = 23)
                            #plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2 / %.2f km / %.2f dB" %(zdr_areas[i[0]].magnitude, zdr_dist[i[0]], zdr_forw[i[0]]), size = 23)
                            #plt.annotate(zdr_areas[i[0]], (zdr_centroid_lon[i[0]],zdr_centroid_lat[i[0]]))
                        #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
                        #plt.tight_layout()
                        #plt.savefig('ZDRarcannotated.png')
                        storm_times = []
                        for l in range(len(max_lons_c)):
                            storm_times.append((time_start))

                    #If there are no storms, set everything to empty arrays!
                    else:
                        print('Filling arrays with no storms')
                        storm_ids = []
                        storm_ids = []
                        max_lons_c = []
                        max_lats_c = []
                        p_zdr_areas = []
                        p_zdr_maxes = []
                        p_zdr_means = []
                        p_zdr_devs = []
                        p_zdr_10max = []
                        p_zdr_mode = []
                        p_zdr_median = []
                        p_separations = []
                        p_sp_angle = []
                        zdr_con_areas1 = []
                        storm_times = time_start
                    #Now record all data in a Pandas dataframe.
                    print('making dataframe')
                    new_cells = pd.DataFrame({
                        'scan': scan_index,
                        'storm_id' : storm_ids,
                        'storm_id1' : storm_ids,
                        'storm_lon' : max_lons_c,
                        'storm_lat' : max_lats_c,
                        'zdr_area' : p_zdr_areas,
                        'zdr_max' : p_zdr_maxes,
                        'zdr_mean' : p_zdr_means,
                        'zdr_std' : p_zdr_devs,
                        'zdr_10max' : p_zdr_10max,
                        'zdr_mode' : p_zdr_mode,
                        'zdr_median' : p_zdr_median,
                        'kdp_zdr_sep' : p_separations,
                        'kdp_zdr_angle' : p_sp_angle,
                        'times' : storm_times
                    })
                    print('setting index')
                    new_cells.set_index(['scan', 'storm_id'], inplace=True)
                    if scan_index == 0:
                        print('first dataframe')
                        tracks_dataframe = new_cells
                    else:
                        tracks_dataframe = tracks_dataframe.append(new_cells)
                    n = n+1
                    scan_index = scan_index + 1
                    #max_lons_p = max_lons_c
                    #max_lats_p = max_lats_c
                    #storm_ids_p = storm_ids
                    #Plot the consolidated stuff!
                    #Write some text objects for the ZDR arc attributes to add to the placefile
                    f.write("Color: 139 000 000 \n")
                    f.write('Font: 1, 30, 1,"Arial" \n')
                    print('wrote font')
                    for y in range(len(p_zdr_areas)):
                        print(y)
                        #f.write('Text: '+str(max_lats_c[y])+','+str(max_lons_c[y])+', 1, "X"," Arc Area: '+str(p_zdr_areas[y])+'\\n Arc Mean: '+str(p_zdr_means[y])+'\\n KDP-ZDR Separation: '+str(p_separations[y])+'\\n Separation Angle: '+str(p_sp_angle[y])+'" \n')
                        f.write('Text: '+str(max_lats_c[y])+','+str(max_lons_c[y])+', 1, "X"," Arc Area: %.2f km^2 \\n Arc Mean: %.2f dB \\n Arc 10 Max Mean: %.2f dB \\n KDP-ZDR Separation: %.2f km \\n Separation Angle: %.2f degrees" \n' %(p_zdr_areas[y], p_zdr_means[y], p_zdr_10max[y], p_separations[y], p_sp_angle[y]))
                    print('made it to plotting')
                    if ((len(zdr_con_areas1) > 0) & (len(max_lons_c) > 0)):
                        #ax.scatter(zdr_con_centroid_lon, zdr_con_centroid_lat, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                        try:
                            for i in enumerate(zdr_con_centroid_lats):
                                print("consolidated ZDR:")
                                ax.scatter(zdr_con_centroid_lons, zdr_con_centroid_lats, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                                try:
                                    plt.text(zdr_con_centroid_lons[i[0]]+.025, zdr_con_centroid_lats[i[0]]+.016, "%.2f km^2 / %.2f dB" %(zdr_con_areas1[i[0]], zdr_con_maxes[i[0]]), size = 23)
                                except:
                                    print("oops zdr")
                            #plt.text(kdp_con_centroid_lon[i[0]]-.20, kdp_con_centroid_lat[i[0]]+.016, "%.2f km" %(dist_kdp_zdr[i[0]]), size = 23, color = 'red')                
                        except:
                            print('failed')
                            try:
                                plt.text(float(zdr_con_centroid_lons)+.016, float(zdr_con_centroid_lats)+.016, "%.2f km^2 / %.2f dB" %(zdr_con_areas1[i[0]], zdr_con_maxes[i[0]]), size = 23)
                            except:
                                print('no zdr centroids')
                            #plt.text(float(kdp_con_centroid_lon)-.20, float(kdp_con_centroid_lat)+.016, "%.2f km" %(float(dist_kdp_zdr[0])), size = 23, color = 'red')
                        if len(kdp_con_areas) > 0:
                            #ax.scatter(zdr_con_centroid_lon, zdr_con_centroid_lat, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                            try:
                                for i in kdp_inds[0]:
                                    #plt.text(zdr_con_centroid_lon[i[0]]+.025, zdr_con_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_con_areas[i[0]].magnitude), size = 23)
                                    try:
                                        plt.text(kdp_con_centroid_lons[i]-.20, kdp_con_centroid_lats[i]+.016, "%.2f km" %(shaped_dist[i]), size = 23, color = 'red')                
                                    except:
                                        print('oops kdp')
                            except:
                                print('failed')
                                #plt.text(float(zdr_con_centroid_lon)+.016, float(zdr_con_centroid_lat)+.016, "%.2f km^2" %(float(zdr_con_areas[0])), size = 23)
                                try:
                                    plt.text(float(kdp_con_centroid_lons)-.20, float(kdp_con_centroid_lats)+.016, "%.2f km" %(float(shaped_dist[0])), size = 23, color = 'red')
                                except:
                                    print('no kdp centroids')
                        else:
                            print('No kdp')
                    else:
                        print('No zdr arcs')
                    print("means there's a kdp problem")
                    #hour = time_start.hour
                    #if hour < 10:
                    #    hour = '0'+str(hour)
                    #minute = time_start.minute
                    #if minute < 10:
                    #    minute = '0'+str(minute)
                    #day = time_start.day
                    #if day < 10:
                    #    day = '0'+str(day)
                    title_plot = plt.title(station+' Radar Reflectivity, ZDR, and KDP '+str(time_start.year)+'-'+str(time_start.month)+'-'+str(time_start.day)+
                                               ' '+str(hour)+':'+str(minute)+' UTC', size = 25)
                    #if np.asarray(zdr_storm_lon).shape[0] > 0:
                    #    ax.scatter(zdr_storm_lon,zdr_storm_lat, marker = "o", color = 'purple', s = 500)
                    #if np.asarray(kdp_storm_lon).shape[0] > 0:
                    #    ax.scatter(kdp_storm_lon,kdp_storm_lat, marker = "o", color = 'purple', s = 500)
                    #try:
                    #    ax.scatter(max_lons_c,max_lats_c, marker = "o", color = 'k', s = 500, alpha = .6)
                    #except:
                    #    "No storm centroids found"
                    try:
                        plt.plot([zdr_con_centroid_lons[kdp_inds], kdp_con_centroid_lons[kdp_inds]], [zdr_con_centroid_lats[kdp_inds],kdp_con_centroid_lats[kdp_inds]], color = 'k', linewidth = 5, transform=ccrs.PlateCarree())
                    except:
                        print('KDP-ZDR separation didt work')
                    ref_centroid_lon = max_lons_c
                    ref_centroid_lat = max_lats_c
                    if len(max_lons_c) > 0:
                        ax.scatter(max_lons_c,max_lats_c, marker = "o", color = 'k', s = 500, alpha = .6)
                        for i in enumerate(ref_centroid_lon): 
                            plt.text(ref_centroid_lon[i[0]]+.016, ref_centroid_lat[i[0]]+.016, "storm_id: %.1f" %(storm_ids[i[0]]), size = 25)
                    #Comment out this line if not plotting tornado tracks
                    #plt.plot([start_torlons, end_torlons], [start_torlats, end_torlats], color = 'purple', linewidth = 5, transform=ccrs.PlateCarree())
                    #Add legend stuff
                    zdr_outline = mlines.Line2D([], [], color='blue', linewidth = 5, linestyle = 'solid', label='ZDR Arc Outline(Area/Max)')
                    kdp_outline = mlines.Line2D([], [], color='green', linewidth = 5,linestyle = 'solid', label='"KDP Foot" Outline')
                    separation_vector = mlines.Line2D([], [], color='black', linewidth = 5,linestyle = 'solid', label='KDP/ZDR Centroid Separation Vector (Red Text=Distance)')
                    #tor_track = mlines.Line2D([], [], color='purple', linewidth = 5,linestyle = 'solid', label='Tornado Tracks')
                    elevation = mlines.Line2D([], [], color='grey', linewidth = 5,linestyle = 'solid', label='Height AGL (m)')

                    plt.legend(handles=[zdr_outline, kdp_outline, separation_vector, elevation], loc = 3, fontsize = 25)
                    alt_levs = [1000, 2000]
                    cele = ax.contour(ungrid_lons,ungrid_lats,gate_altitude-radar.altitude['data'][0],alt_levs, linewidths = 7, alpha = .6, colors = 'grey')
                    plt.clabel(cele, fontsize=18, inline=1, inline_spacing=10, fmt='%i', rightside_up=True, use_clabeltext=True)
                    plt.tight_layout()
                    print('made it to saving')
                    plt.savefig('Machine_Learning/ZSlideR325P_ZDRArc_example'+station+str(time_start.year)+str(time_start.month)+str(day)+str(hour)+str(minute)+'.png')
                    print('figure saved')
                    plt.close()

        except:
            continue
    f.close()
    plt.show()
    return tracks_dataframe

In [5]:
#Loop to run the actual algorithm
print(datetime.utcnow())
#You can either run all example cases (commented out line) or one at a time (runing line)
#for i in range(len(durationstm)):
for i in [0]:
    tracks_dataframe = pd.DataFrame()
    tracks_dataframe = multi_case_algorithm_ML1(storm_relative_dirstm[i], zdrlevstm[i], kdplevstm[i], REFlevstm[i], REFlev1stm[i], big_stormstm[i], zero_z_triggerstm[i], storm_to_trackstm[i], yearstm[i], monthstm[i], daystm[i], hourstm[i], start_minstm[i], durationstm[i], stationstm[i])
    #Save the dataframe off as a pickle file
    tracks_dataframe.to_pickle('MLexample_valid_tor'+str(yearstm[i])+str(monthstm[i])+str(daystm[i])+str(stationstm[i])+'.pkl')
print(datetime.utcnow())

2019-08-01 19:55:15.826813
Downloaded KPOE20170402_173137_V06
1 out of 1 files downloaded...0 errors


C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.0 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


in loop
23
0.41806
2017-04-02 17:31:37.003000
its this line
heres the problem
2019-08-01 19:55:37.023559
2019-08-01 19:55:37.077529


C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\numpy\ma\core.py:4185: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


2019-08-01 19:56:19.377360
almost gridding


...

2019-08-01 19:58:49.774767
almost gridding


C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\pyart\map\gates_to_grid.py:177: DeprecationWarning: Barnes weighting function is deprecated. Please use Barnes 2 to be consistent with Pauley and Wu 1990.
  " Pauley and Wu 1990.", DeprecationWarning)
C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\numpy\ma\core.py:852: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)


2019-08-01 19:58:50.664259
made it to smoothing
made it through gradient
[[-- -- -- ..., -- -- --] [-- -- -- ..., -- -- --] [-- -- -- ..., -- -- --] ...,  [-- -- -- ..., 0.1866157054901123 0.12651562690734863 0.09039306640625] [-- -- -- ..., 0.2915985584259033 0.20787476003170013 0.15343499183654785] [-- -- -- ..., 0.41399598121643066 0.30831098556518555 0.23376131057739258]] radian
3.1413204669952393 radian
5.4836273193359375e-06 radian
[[  0.           0.           0.         ...,   0.           0.           0.        ] [  0.           0.           0.         ...,   0.           0.           0.        ] [  0.           0.           0.         ...,   0.           0.           0.        ] ...,  [  0.           0.           0.         ...,  10.69229221   7.24881172    5.17914104] [  0.           0.           0.         ...,  16.70736694  11.91034698    8.79117775] [  0.           0.           0.         ...,  23.72022247  17.6649189   13.39353657]] degree
179.98440551757812 degree
0.0 d

C:\Users\matts\Miniconda3\envs\devnew\lib\site-packages\ipykernel_launcher.py:902: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


[]
(1, -93.860105669454541)
[]
(2, -92.769084879832121)
[0]
(3, -92.730960942431437)
[]
(4, -93.123072080905004)
[1]
(5, -92.785992038591871)
[]
(6, -94.03907355282945)
[]
(7, -94.148869758432383)
[]
made it through giant if statement
plotted centroids
making dataframe
setting index
wrote font
0
1
2
3
4
5
6
7
made it to plotting
consolidated ZDR:
consolidated ZDR:
means there's a kdp problem
made it to saving
figure saved
in loop
23
in loop
23
in loop
23
in loop
23
in loop
23
2019-08-01 19:59:01.816580


You should now have three different forms of algorithm output:
    1. Saved .png images of each radar scan.
    2. A looping GR2 placefile of all algorithm output.
    3. A Pickle file containing tracks for all storms and their associated ZDR arc characteristics.